In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv('../Weather_Merged_CSVs/Banana.csv')
df.head()

,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Price Date,Day Of Week,lookback_temp_mean,lookback_precip_sum
0,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4000.0,4500.0,4500.0,2025-06-18,2,27.779167,258.2
1,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4000.0,4500.0,4500.0,2024-12-26,3,25.363333,578.0
2,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4500.0,5000.0,5000.0,2025-06-01,6,27.277500,217.9
3,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4500.0,5000.0,5000.0,2025-06-17,1,27.753333,256.6
4,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4500.0,5000.0,5000.0,2024-11-18,0,26.224167,542.6


In [11]:
# Time Series Analysis 
df['Price Date'] = pd.to_datetime(df['Price Date'])
df.sort_values('Price Date', inplace=True)
df.set_index('Price Date', inplace=True)
df.head()

,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Day Of Week,lookback_temp_mean,lookback_precip_sum
Price Date,,,,,,,,,,,
2016-04-30,Coimbatore,Sulur,Banana,Other,Large,1200.0,1400.0,1300.0,5,27.207500,12.8
2018-05-07,Coimbatore,Sulur,Banana,Other,Large,3200.0,3400.0,3300.0,0,27.240833,117.6
2018-09-20,Theni,Chinnamanur,Banana,Poovan,Medium,1300.0,1600.0,1450.0,3,26.824167,309.3
2018-09-27,Theni,Chinnamanur,Banana,Poovan,Medium,1300.0,1600.0,1450.0,3,26.780833,359.3
2018-10-12,Theni,Chinnamanur,Banana,Other,Medium,1200.0,1400.0,1300.0,4,26.440833,505.7


In [12]:
categorical_cols = ['District Name', 'Market Name', 'Commodity', 'Variety', 'Grade']
for col in categorical_cols:
    encoder = LabelEncoder()
    # Fit the encoder to the column and transform the text to numbers (e.g., 'Coimbatore', 'Theni' -> 0, 1)
    df[col] = encoder.fit_transform(df[col])
df.head()

,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Day Of Week,lookback_temp_mean,lookback_precip_sum
Price Date,,,,,,,,,,,
2016-04-30,2,191,0,1,0,1200.0,1400.0,1300.0,5,27.207500,12.8
2018-05-07,2,191,0,1,0,3200.0,3400.0,3300.0,0,27.240833,117.6
2018-09-20,24,32,0,2,2,1300.0,1600.0,1450.0,3,26.824167,309.3
2018-09-27,24,32,0,2,2,1300.0,1600.0,1450.0,3,26.780833,359.3
2018-10-12,24,32,0,1,2,1200.0,1400.0,1300.0,4,26.440833,505.7
